In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [2]:
dfTrain = pd.read_csv('travel_insurance/train.csv')
dfTest = pd.read_csv('travel_insurance/test.csv')

In [3]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590 entries, 0 to 1589
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Customer Id          1590 non-null   object
 1   Age                  1590 non-null   int64 
 2   Employment Type      1590 non-null   object
 3   GraduateOrNot        1590 non-null   object
 4   AnnualIncome         1590 non-null   int64 
 5   FamilyMembers        1590 non-null   int64 
 6   ChronicDiseases      1590 non-null   int64 
 7   FrequentFlyer        1590 non-null   object
 8   EverTravelledAbroad  1590 non-null   object
 9   TravelInsurance      1590 non-null   object
dtypes: int64(4), object(6)
memory usage: 124.3+ KB


In [4]:
dfTrain.head()

,Customer Id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,3JUN0VW6F043,34,Private Sector/Self Employed,Yes,1300000,6,0,Yes,No,No
1,VLHY2ABIR4QL,28,Private Sector/Self Employed,Yes,750000,7,0,Yes,No,No
2,6E3F7UNXYNFF,28,Private Sector/Self Employed,Yes,750000,6,0,Yes,No,No
3,JJ8R0ZRYWR31,32,Government Sector,Yes,800000,6,1,No,No,No
4,2WGFUEX6IEHM,34,Private Sector/Self Employed,Yes,700000,4,1,No,No,No


In [5]:
dfTrain["AnnualIncome"].sum()/1590

928553.4591194969

In [6]:
dfTrain["FrequentFlyer"]

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
1585     No
1586     No
1587     No
1588     No
1589    Yes
Name: FrequentFlyer, Length: 1590, dtype: object

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Yes', 'No'])

def dfColChange(df, test=0):
    df = df.copy()
    df["Employment Type"] = df["Employment Type"].apply(lambda x: "Yes" if x=="Private Sector/Self Employed" else "No")
    cols = ["GraduateOrNot", "EverTravelledAbroad", "FrequentFlyer", "Employment Type"]
    if not test: cols.append("TravelInsurance")
    for col in cols:
        df[col] = le.transform(df[col])
    
    return df

In [8]:
from sklearn.ensemble import RandomForestClassifier

df = dfColChange(dfTrain)
X, y = df[["Age","Employment Type","GraduateOrNot","AnnualIncome","FamilyMembers", "ChronicDiseases",
        "FrequentFlyer","EverTravelledAbroad"]] , df["TravelInsurance"]

clf = RandomForestClassifier(n_estimators=400, random_state=0)
clf.fit(X,y)

RandomForestClassifier(n_estimators=400, random_state=0)

In [9]:
cols = ["Age","Employment Type","GraduateOrNot","AnnualIncome","FamilyMembers","ChronicDiseases",
        "FrequentFlyer","EverTravelledAbroad"]
test = dfColChange(dfTest, 1)
yhat = clf.predict_proba(test[cols])

newdf = pd.concat([test['Customer Id'], pd.Series(yhat[:,1])], axis=1, keys=['Customer Id', 'prediction'])
newdf.to_csv('output2.csv', index=False)
newdf

,Customer Id,prediction
0,Q5OK4L2Q5XNH,0.898532
1,CBHVMVE7TRP5,1.000000
2,9UGHQ91U4F0M,0.323345
3,3UWXWMZK4ZGL,0.982500
4,5CGEMLMBOPES,0.089534
...,...,...
392,3T4IU1ZJNDIA,0.457060
393,N3ON6CQWDU4Z,0.494338
394,LWW08WASNJ2T,0.086704
395,L2RAKTUJP4HS,0.015292
